In [5]:
#lab06-01
'''
y hat ->expectation
multinomial classification: 여러 개의 class가 있음
softmax: 모든 값이 0~1이고 다 더하면 1이 된다.(각각을 확률로 본다.)
one hot encoding: 가장 큰 값을 1로 해주고 나머지는 0으로 바꿔줌
cost function: 실제값과 예측값의 차이
cross entropy: ylog(h(x))-(1-y)log(1-h((x)) 실제 값과 예측 값의 차이
'''
import tensorflow as tf

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X = tf.placeholder('float', [None, 4]) #X = input이 x1~x4
Y = tf.placeholder('float', [None, 3])#Y = class의 개수와 같다(label의 개수), 따라서 3개
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis = 1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

#session을 열고 초기화, loop을 돌면서 학습시켜줌
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict = {X: x_data, Y: y_data}))

    a = sess.run(hypothesis, feed_dict = {X: [[1,11,7,9]]})
    print('a', a, sess.run(tf.arg_max(a,1)))
    
    b = sess.run(hypothesis, feed_dict = {X: [[1,11,7,9]]})
    print('b', b, sess.run(tf.arg_max(a,1)))
    
    c = sess.run(hypothesis, feed_dict = {X: [[1,11,7,9]]})
    print('c', c, sess.run(tf.arg_max(a,1)))

    all = sess.run(hypothesis, feed_dict = {X: [[1,11,7,9],
                                         [1,3,4,3],
                                         [1,1,0,1]]})
    print('all', all, sess.run(tf.arg_max(all,1)))
#arg_max 몇 번째에 있는 argument가 가장 높은가요를 알려줌.
#[1]을 돌려준다.
#여러 개의 경우를 질문할 수 있다. all->[1,11,7,9]-1번째 경우는 어떻니
#9.99665856e-01: 0.9이다.



0 3.8169532
200 0.5443719
400 0.41871905
600 0.3171831
800 0.25338995
1000 0.22674817
1200 0.20535687
1400 0.18768516
1600 0.17279908
1800 0.16007304
2000 0.14906392
a [[1.4608122e-03 9.9853253e-01 6.7002534e-06]] [1]
b [[1.4608122e-03 9.9853253e-01 6.7002534e-06]] [1]
c [[1.4608122e-03 9.9853253e-01 6.7002534e-06]] [1]
all [[1.4608108e-03 9.9853253e-01 6.7002534e-06]
 [9.2777389e-01 6.8465866e-02 3.7602640e-03]
 [7.6764683e-09 2.9171834e-04 9.9970824e-01]] [1 0 2]


In [2]:
import tensorflow as tf
import numpy as np

xy = np.loadtxt('data-04-zoo.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
nb_classes = 7 #0~6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])

Y_one_hot = tf.one_hot(Y, nb_classes)
#y값, 몇 개의 class가 있는지 알려줌
#rank가 1이면 output은 1+1이다. 차원이 하나 더 늘어남
#[[0],[3]]---one_hot--->[[[1000000]]][[[0001000]]] (2차원인데 3차원이 됨)
#이럴 때 간단하게 하는 것이 reshape임
#우리가 원하는 shape는 (?,7)임
#[[[1000000]]][[[0001000]]]---reshape--->[[1000000],[0001000]]
#우리가 처음 준 one_hot의 모양대로 나옴

print('one_hot: ', Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
#
print('reshape one_hot', Y_one_hot)

W = tf.Variable(tf.random_normal([16,nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits,
                                                labels = Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
#예측한 값이 맞는지 틀린지 보고 싶다. 
#prediction: hypothesis의 확률에 argmax를 취해서 hypothesis를 level로 바꾸겠다
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#그 probability를 0-6 사이의 값 중 하나로 만들어내는 것
#one hot중 하나를 택한 값인 label과 비교

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2000):
        sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict = {
                X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    pred = sess.run(prediction, feed_dict = {X: x_data})
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))
    
    #추가로 내가 만든 prediction
    pred2 = sess.run(prediction, feed_dict = {X: [[ 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 2., 0., 1., 1.]]})
    print(pred2)
#x_data를 하나 던져주고 predict해본다.
#flatten: [[1],[0]] -> [1,0]
#zip: 각각의 element를 넘겨주기 편하게 하기 위해 zip으로 만들겠다.
#우리가 예측한 값이 맞는지 true/false, predict한 값, 원래 y의 값을 출력
#loop을 돌면서 loss가 작아짐


one_hot:  Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Step:     0	Loss: 4.365	Acc: 8.91%
Step:   100	Loss: 0.514	Acc: 88.12%
Step:   200	Loss: 0.363	Acc: 91.09%
Step:   300	Loss: 0.290	Acc: 94.06%
Step:   400	Loss: 0.243	Acc: 96.04%
Step:   500	Loss: 0.208	Acc: 97.03%
Step:   600	Loss: 0.181	Acc: 97.03%
Step:   700	Loss: 0.159	Acc: 97.03%
Step:   800	Loss: 0.141	Acc: 97.03%
Step:   900	Loss: 0.127	Acc: 97.03%
Step:  1000	Loss: 0.114	Acc: 99.01%
Step:  1100	Loss: 0.104	Acc: 100.00%
Step:  1200	Loss: 0.095	Acc: 100.00%
Step:  1300	Loss: 0.088	Acc: 100.00%
Step:  1400	Loss: 0.082	Acc: 100.00%
Step:  1500	Loss: 0.076	Acc: 100.00%
Step:  1600	Loss: 0.071	Acc: 100.00%
Step:  1700	Loss: 0.067	Acc: 100.00%
Step:  1800	Loss: 0.064	Acc: